## Webscrapper Jobs: Monster

In [1]:
import requests
from bs4 import BeautifulSoup

job_type = "education"
loc = "texas"

URL = f"https://www.monster.com/jobs/search/?q={job_type}\
        &where= {loc}"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id="ResultsContainer")

# Look for Python jobs
python_jobs = results.find_all("h2", string=lambda t: "python" in t.lower())
for p_job in python_jobs:
    link = p_job.find("a")["href"]
    print(p_job.text.strip())
    print(f"Apply here: {link}\n")

# Print out all available jobs from the scraped webpage
job_elems = results.find_all("section", class_="card-content")
for job_elem in job_elems:
    title_elem = job_elem.find("h2", class_="title")
    company_elem = job_elem.find("div", class_="company")
    location_elem = job_elem.find("div", class_="location")
    if None in (title_elem, company_elem, location_elem):
        continue
    print(title_elem.text.strip())
    print(company_elem.text.strip())
    print(location_elem.text.strip())
    print()

RN, Clinical Educator - Nursing Education
CHRISTUS Health
Texarkana, TX

Director, Clinical Education Program Surgical Tech - Baptist School of Health Professions
School of Health Professions
San Antonio, TX

Project Architect - K12/Higher Education
CyberCoders
Austin, TX

Admissions Coordinator
SavaSeniorCare
Houston, TX

Child Care Teacher (TX)
Childcare Network
Frisco, TX

Teacher
Headstart of Greater Dallas, Inc.
Dallas, TX

Teachers
Company Confidential
HOUSTON, TX

ASSISTANT INSTRUCTOR, DENTAL ASSISTING
Del Mar College
Corpus Christi, TX

Coach
Renew Fitness
AUSTIN, TX

Coach
Paris Junior College
Paris, TX

REMOTE Sr. DevOps Engineer
Applied Resource Group


Full-Time Welding Instructor, Huntsville Center
Lee College
Baytown, TX

Parts Trainer - JO 43541
Resource Employment Solutions
Grand Prairie, TX

Instrumentation Technology Faculty- Program Director
Galveston College
Galveston, TX

Academic Teacher-Statewide
Windham Texas School District
Huntsville, TX

Mental Health Intake 

## Webscrapper Jobs: Seven Days

In [7]:
import requests
from bs4 import BeautifulSoup

emp_type = "education"
loc = "any"

URL = f"https://jobs.sevendaysvt.com/browse-jobs/?search_keywords={emp_type}&search_location={any}#.X0UFWElOnOQ"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id="container")

# Look for jobs
jobs = results.find_all("h2", string=lambda t: "data" in t.lower())
for job in jobs:
    link = job.find("a")["href"]
    print(job.text.strip())
    print(f"Apply here: {link}\n")

# Print out all available jobs from the scraped webpage
job_elems = results.find_all("section", class_="card-content")
for job_elem in job_elems:
    title_elem = job_elem.find("h2", class_="title")
    company_elem = job_elem.find("div", class_="company")
    location_elem = job_elem.find("div", class_="location")
    if None in (title_elem, company_elem, location_elem):
        continue
    print(title_elem.text.strip())
    print(company_elem.text.strip())
    print(location_elem.text.strip())
    print()

AttributeError: 'NoneType' object has no attribute 'find_all'